# Быстрое обучение модели

Мы собрали данные по докторам, теперь мы можем обучить простую модель для предсказания стоимости приёма

In [3]:
import json
import numpy as np


Загрузим данные из JSON-a

In [5]:
with open('kinopoisk.json', 'r') as f:
    kinopoisk = json.load(f)

Проверим содержимое объекта

In [6]:
kinopoisk[0]

{'assessment': 1,
 'author': 'FRIDMON',
 'date': '13 февраля 2018',
 'film': 'Камила',
 'review': '\n      «Ребёнок стоит ближе меня, ближе каждого взрослого человека к\xa0тому идеалу гармонии, красоты и\xa0добра, до\xa0которого я\xa0в своей гордости хочу возвести его». (Лев Толстой)\n\nЕсть фильмы, которые невозможно «разложить по\xa0полочкам», потому что\xa0душа человеческая\xa0—\xa0это не\xa0библиотечный стеллаж. Вот\xa0и работу Мухтара Ага-Мирзаева очень сложно описать словами, потому что\xa0вся она\xa0от начала до\xa0конца букально пропитана музыкой. Музыка здесь повсюду. Наравне с\xa0семилетней Камилой она\xa0является главным персонажем картины. Остаётся лишь удивляться, как\xa0режиссёру удалось столь тонко, поистине филигранно вплести музыку в\xa0канву повествования. Разумеется, это\xa0заслуга не\xa0одного режиссёра. Искренняя благодарность всем без\xa0исключения создателям картины\xa0—\xa0сценаристам Камилу Икрамову и\xa0Ольге Сидельниковой за\xa0историю, в\xa0которой нет\xa0ни

Создадим выборку для обучения

Удобно в комментариях писать название признака

In [7]:
X, y = [], []

for kino in kinopoisk:
    features = []
    # 0 - author
    features.append(kino['author'])
    # 1 - date
    features.append(kino['date'])
    # 2 - film
    features.append(kino['film'])
    # 3 - review
    features.append(kino['review'])
    # 4 - 87 one hot proffesions
    
    X.append(features)
    y.append(kino['assessment'] or 0)
    
X = np.array(X)
y = np.array(y)

In [8]:
print(len(X), len(y), type(X))
print(X[0][3])

548 548 <class 'numpy.ndarray'>

      «Ребёнок стоит ближе меня, ближе каждого взрослого человека к тому идеалу гармонии, красоты и добра, до которого я в своей гордости хочу возвести его». (Лев Толстой)

Есть фильмы, которые невозможно «разложить по полочкам», потому что душа человеческая — это не библиотечный стеллаж. Вот и работу Мухтара Ага-Мирзаева очень сложно описать словами, потому что вся она от начала до конца букально пропитана музыкой. Музыка здесь повсюду. Наравне с семилетней Камилой она является главным персонажем картины. Остаётся лишь удивляться, как режиссёру удалось столь тонко, поистине филигранно вплести музыку в канву повествования. Разумеется, это заслуга не одного режиссёра. Искренняя благодарность всем без исключения создателям картины — сценаристам Камилу Икрамову и Ольге Сидельниковой за историю, в которой нет ни одной лишней минуты. Оператору Хатаму Файзиеву, сумевшему через оптику кинокамеры донести до зрителей внутренний мир девочки. Композитору Румилю Ви

In [9]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(
    analyzer = 'word',
    lowercase = False,
)
features = vectorizer.fit_transform(X[:,3])
features_nd = features.toarray() # for easy usage

In [10]:
from sklearn.cross_validation import train_test_split
 
X_train, X_test, y_train, y_test  = train_test_split(
        features_nd, 
        y,
        train_size=0.80, 
        random_state=1234)

Обучим простые модели

In [11]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.dummy import DummyRegressor

DummyRegressor это простой бейзлайн - предсказание константой

In [31]:
np.sqrt(-cross_val_score(DummyRegressor(), X_train, y_train, cv=3, scoring='neg_mean_squared_error').mean())

0.61288932694104825

In [32]:
np.sqrt(-cross_val_score(LinearRegression(), X_train, y_train, cv=3, scoring='neg_mean_squared_error').mean())

0.61288932694104825

In [33]:
np.sqrt(-cross_val_score(RandomForestRegressor(), X_train, y_train, cv=3, scoring='neg_mean_squared_error').mean())

0.61288935883445983

In [34]:
np.sqrt(-cross_val_score(RandomForestRegressor(n_estimators=100), X_train, y_train, cv=3, scoring='neg_mean_squared_error').mean())

0.61292445748474889

In [35]:
np.sqrt(-cross_val_score(KNeighborsRegressor(), X_train, y_train, cv=3, scoring='neg_mean_squared_error').mean())

0.73173341453997975

In [36]:
np.sqrt(-cross_val_score(KNeighborsRegressor(n_neighbors=10), X_train, y_train, cv=3, scoring='neg_mean_squared_error').mean())

0.72732072278325355

In [1]:
import xgboost as xgb

/Users/Timur/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [12]:
np.sqrt(-cross_val_score(xgb.XGBRegressor(), X_train, y_train, cv=3, scoring='neg_mean_squared_error').mean())

0.61288932272326102

xgboost лучше всех, посмотрим на самые важность признаков

In [ ]:
algo = xgb.XGBRegressor().fit(X_train, y_train)
sorted(zip(algo.feature_importances_, range(X.shape[1])), reverse=True)

In [ ]:
sorted(zip(algo.feature_importances_, range(X.shape[1])), reverse=True)[:7]

0 - experience

3 - len(proffesions)

2 - is_phd

Сохраним модель

In [15]:
algo.booster().save_model('xgboost_model')

Загрузим модель с диска и проверим, что всё корректно

In [25]:
bst = xgb.Booster()
bst.load_model('xgboost_model')

In [26]:
clf = xgb.XGBRegressor()
booster = xgb.Booster()
booster.load_model('xgboost_model')
clf._Booster = booster

In [27]:
clf.predict(X)

array([2006.3156, 2191.837 , 2861.2122, ..., 2339.8147,  565.0743,
       1137.5759], dtype=float32)

In [28]:
algo.predict(X)

array([2006.3156, 2191.837 , 2861.2122, ..., 2339.8147,  565.0743,
       1137.5759], dtype=float32)

In [ ]:
# save the model to disk
filename = 'finalized_model.sav'
pickle.dump(model, open(filename, 'wb'))
 
# some time later...
 
# load the model from disk
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.score(X_test, Y_test)
print(result)